# CAI - Rebalancing data for November 2022 

## Setting

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import time
import os
from decouple import config

In [2]:
from pycoingecko import CoinGeckoAPI
key = config("CG_KEY")
cg = CoinGeckoAPI(api_key= key)

In [3]:
import plotly.express as px
import plotly.graph_objects as go

In [4]:
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

## Data retrieving and filtering

### Token inclusion criteria

1. **Project and token characteristics**
    1. Any token included in our index must meet all the criteria below:
    1. The project’s token should have been listed on CoinGecko with pricing data at least 6 months prior to the date of inclusion in the index.
    1. The project should have a token that is native to Avalanche. This excludes wrapped variants, where the underlying tokens are locked on an alt-L1.
    1. The project should be a going concern, with a dedicated team actively building, supporting and maintaining the project.
    1. No rebasing or deflationary tokens.
    1. The project must be widely considered to be building a useful protocol or product. Projects that have ponzi characteristics at the core of their offering will not be considered.
    1. Synthetic tokens which derive their value from external price feeds are not permissible.
    1. The project’s token must not have the ability to pause token transfers.
    1. The project’s protocol or product must have significant usage.
    <br>
1. **Pricing requirements**
    1. Token should have a Chainlink price feed to allow for manipulation resistant pricing.
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 30% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange. 
    1. The token should have in aggregate at least $2mm of onchain liquidity across Trader Joe, Platypus, Pangolin.
    1. The token must have shown consistent DeFi liquidity on Avalanche.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.

### Tokens list

In [5]:
def normalize_symb(symb):
    return ''.join(c for c in symb if c.isalnum()).upper()

#### CoinGecko

##### Tokens addresses

In [6]:
coins_list = cg.get_coins_list(include_platform=True)
coins = {}
for coin in coins_list:
    if 'avalanche' in coin['platforms']:
        coin['addr'] = coin['platforms']['avalanche']
        coin['symbol'] = normalize_symb(coin['symbol'])
        coins[coin['id']] = coin

In [7]:
# wrapper avax
coins['avalanche-2']['addr'] = "0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7"

##### Market data

In [8]:
cat_id = "avalanche-ecosystem"
cg_tokens_data = cg.get_coins_markets(vs_currency='USD', order='market_cap_desc', category=cat_id)
for i in range(2, 30):
    cg_tokens_data.extend(cg.get_coins_markets(vs_currency='USD', order='market_cap_desc', category=cat_id, page=i))
    
cg_tokens_data = pd.DataFrame.from_records(cg_tokens_data)

In [9]:
tokens_market_data = cg_tokens_data.set_index('id')[
    ['name', 'symbol', 'market_cap', 'market_cap_rank', 'current_price', 'total_supply', 'circulating_supply']
]
tokens_market_data['symbol'] = tokens_market_data['symbol'].apply(normalize_symb)

In [10]:
tokens_market_data = tokens_market_data[tokens_market_data['market_cap'] > 0]
tokens_market_data = tokens_market_data[~tokens_market_data.index.duplicated(keep='first')]

In [11]:
tokens_market_data = tokens_market_data.join(pd.DataFrame.from_dict(coins, orient='index')['addr'])
# tokens_market_data = tokens_market_data[~tokens_market_data['addr'].isna()]

In [12]:
tokens_market_data = tokens_market_data.sort_values('market_cap', ascending=False)

#Adjusting Pangolin's max supply to reflect tokenomic changes here: https://app.pangolin.exchange/#/vote/10
tokens_market_data.loc[['pangolin'],["total_supply"]] = 230000000.0

### Token's supply check 

In [13]:
sp_passed_index = (tokens_market_data['circulating_supply'] / tokens_market_data['total_supply']) > 0.3
sp_passed = tokens_market_data[sp_passed_index]

In [14]:
sp_passed

,name,symbol,market_cap,market_cap_rank,current_price,total_supply,circulating_supply,addr
id,,,,,,,,
tether,Tether,USDT,6.887833e+10,3.0,1.000000,6.888729e+10,6.888729e+10,0x9702230a8ea53601f5cd2dc00fdbc13d4df4a8c7
binancecoin,BNB,BNB,4.865796e+10,4.0,298.060000,1.632770e+08,1.632770e+08,NaN
usd-coin,USD Coin,USDC,4.388859e+10,5.0,1.000000,4.387861e+10,4.387840e+10,0xb97ef9ef8734c71904d8002f8b6bc66dd9c48a6e
binance-usd,Binance USD,BUSD,2.120612e+10,7.0,0.999100,2.122672e+10,2.122672e+10,0x9c9e5fd8bbc25984b178fdce6117defa39d2db39
dai,Dai,DAI,5.766821e+09,16.0,1.000000,5.767306e+09,5.767306e+09,0xd586e7f844cea2f87f50152665bcbc2c279d8d70
...,...,...,...,...,...,...,...,...
cycle-token,Cycle,CYCLE,2.305600e+04,3611.0,0.116964,3.000000e+05,1.978129e+05,0x81440c939f2c1e34fc7048e518a637205a632a74
nft-soccer-games,NFT Soccer Games,NFSG,2.171500e+04,3621.0,0.243261,2.000000e+05,8.928850e+04,0xf69c2fcd9128d49dfa22348c69177f9380438eb8
ape-in,Ape In,APEIN,1.953299e+04,3643.0,0.520880,3.750000e+04,3.750000e+04,0x938fe3788222a74924e062120e7bfac829c719fb


### Liquidity check

In [15]:
api_urls = {
    'trader_joe': "https://api.thegraph.com/subgraphs/name/traderjoe-xyz/exchange",
    'pangolin': "https://api.thegraph.com/subgraphs/name/pangolindex/exchange",
} 

In [16]:
tokens_ids = list(tokens_market_data['addr'].dropna())

In [17]:
gecko_ids = pd.Series(tokens_market_data.index, index=tokens_market_data['addr'].str.lower())
gecko_ids = gecko_ids.groupby(gecko_ids.index).first()

In [18]:
def trader_joe_liqs(tokens_ids):
    query = """
        query pairs($tokens_ids: [ID!]!) {
            pairs (first:1000, 
                where: {
                    token0_in: $tokens_ids, 
                    token1_in: $tokens_ids,
                    reserveUSD_gt: 10000
                }){
                token0 {
                    id
                    symbol
                }
                token1 {
                    id
                    symbol
                }
                reserveUSD
            }
        }
    """
    url = api_urls['trader_joe']
    
    response = requests.post(url, json={'query': query, 'variables': {'tokens_ids': tokens_ids}})
    pairs = response.json()['data']['pairs']
    
    liqs = {}
    for pair in pairs:
        liq = float(pair['reserveUSD'])
        
        id0 = gecko_ids[pair['token0']['id'].lower()]
        liqs[id0] = liqs.get(id0, 0) + liq
        
        id1 = gecko_ids[pair['token1']['id'].lower()]
        liqs[id1] = liqs.get(id1, 0) + liq 
        
    return liqs
    

In [19]:
def pangolin_liqs(tokens_ids):
    query = """
        query pairs($tokens_ids: [ID!]!) {
            pairs (first:1000, 
                where: {
                    token0_in: $tokens_ids, 
                    token1_in: $tokens_ids,
                    reserveUSD_gt: 10000
                }){
                token0 {
                    id
                    symbol
                }
                token1 {
                    id
                    symbol
                }
                reserveUSD
            }
        }
    """
    url = api_urls['pangolin']
    
    response = requests.post(url, json={'query': query, 'variables': {'tokens_ids': tokens_ids}})
    pairs = response.json()['data']['pairs']
    
    liqs = {}
    for pair in pairs:
        liq = float(pair['reserveUSD'])
        
        id0 = gecko_ids[pair['token0']['id'].lower()]
        liqs[id0] = liqs.get(id0, 0) + liq
        
        id1 = gecko_ids[pair['token1']['id'].lower()]
        liqs[id1] = liqs.get(id1, 0) + liq 
        
    return liqs
    

In [20]:
query_functions = {
    'trader_joe': trader_joe_liqs,
    'pangolin': pangolin_liqs
}

In [21]:
liqs_dict = {}
for dex, func in query_functions.items():
    liqs_dict[dex] = func(tokens_ids)

In [22]:
liquidities = pd.DataFrame.from_dict(liqs_dict)

In [23]:
liquidities['sum'] = liquidities.sum(axis=1)
liquidities = liquidities.sort_values('sum', ascending=False)
liquidities.head(20)

,trader_joe,pangolin,sum
avalanche-2,5.419503e+07,2.293833e+07,7.713336e+07
usd-coin,1.820667e+07,8.445594e+06,2.665226e+07
tether,8.704234e+06,3.562653e+06,1.226689e+07
joe,8.314122e+06,1.078473e+04,8.324907e+06
bitcoin-avalanche-bridged-btc-b,6.316378e+06,NaN,6.316378e+06
benqi-liquid-staked-avax,5.117644e+06,3.807213e+05,5.498365e+06
benqi,3.347148e+04,3.443403e+06,3.476875e+06
gmx,2.656127e+06,NaN,2.656127e+06
pangolin,NaN,2.368938e+06,2.368938e+06
chainlink,4.830651e+05,1.388270e+06,1.871335e+06


In [24]:
liq_check = (liquidities['sum'] > 2e6).sort_values(ascending=False)
liq_check_passed = liq_check[liq_check]

In [25]:
lq_passed = sp_passed[
    sp_passed.index.isin(liq_check_passed.index)
]

In [26]:
lq_passed

,name,symbol,market_cap,market_cap_rank,current_price,total_supply,circulating_supply,addr
id,,,,,,,,
tether,Tether,USDT,6.887833e+10,3.0,1.000000,6.888729e+10,6.888729e+10,0x9702230a8ea53601f5cd2dc00fdbc13d4df4a8c7
usd-coin,USD Coin,USDC,4.388859e+10,5.0,1.000000,4.387861e+10,4.387840e+10,0xb97ef9ef8734c71904d8002f8b6bc66dd9c48a6e
avalanche-2,Avalanche,AVAX,5.368785e+09,17.0,17.970000,4.129870e+08,2.982377e+08,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
gmx,GMX,GMX,3.425396e+08,120.0,42.270000,8.168608e+06,8.168585e+06,0x62edc0692bd897d2295872a9ffcac5425011c661
benqi-liquid-staked-avax,BENQI Liquid Staked AVAX,SAVAX,8.824765e+07,275.0,18.810000,4.691643e+06,4.691307e+06,0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be
joe,JOE,JOE,8.634458e+07,279.0,0.252443,4.396521e+08,3.419847e+08,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd
bitcoin-avalanche-bridged-btc-b,Bitcoin Avalanche Bridged (BTC.b),BTCB,8.113133e+07,297.0,20630.000000,3.933019e+03,3.933019e+03,0x152b9d0fdc40c096757f570a51e494bd4b943e50
benqi,BENQI,QI,2.680867e+07,591.0,0.008875,7.200000e+09,3.024486e+09,0x8729438eb15e2c8b576fcc6aecda6a148776c0f5
pangolin,Pangolin,PNG,1.078866e+07,875.0,0.081744,2.300000e+08,1.325136e+08,0x60781c2586d68229fde47564546784ab3faca982


### Non quantative checks

In [27]:
exclude_list = [
    {
        'gecko_id': 'tether',
        'reason': 'stablecoin'
    },
    {
        'gecko_id': 'usd-coin',
        'reason': 'stablecoin'
    },
    {
        'gecko_id': 'spell-token',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'stacktical',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'zookeeper',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'pendle',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'everrise',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'gmx',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'pangolin',
        'reason': 'No manipulation resistant price feed'
    }
]
exclude_ids = [token['gecko_id'] for token in exclude_list]

In [28]:
nq_passed = lq_passed[
    ~lq_passed.index.isin(exclude_ids)
]

In [29]:
nq_passed

,name,symbol,market_cap,market_cap_rank,current_price,total_supply,circulating_supply,addr
id,,,,,,,,
avalanche-2,Avalanche,AVAX,5.368785e+09,17.0,17.970000,4.129870e+08,2.982377e+08,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
benqi-liquid-staked-avax,BENQI Liquid Staked AVAX,SAVAX,8.824765e+07,275.0,18.810000,4.691643e+06,4.691307e+06,0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be
joe,JOE,JOE,8.634458e+07,279.0,0.252443,4.396521e+08,3.419847e+08,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd
bitcoin-avalanche-bridged-btc-b,Bitcoin Avalanche Bridged (BTC.b),BTCB,8.113133e+07,297.0,20630.000000,3.933019e+03,3.933019e+03,0x152b9d0fdc40c096757f570a51e494bd4b943e50
benqi,BENQI,QI,2.680867e+07,591.0,0.008875,7.200000e+09,3.024486e+09,0x8729438eb15e2c8b576fcc6aecda6a148776c0f5


### Retrieve historical data & token age check

In [30]:
import time

In [31]:
exclude_list = []

prices_data = pd.DataFrame()
marketcaps = pd.DataFrame() 
for id_, row in nq_passed.iterrows():
    symbol = row['symbol']
    data = cg.get_coin_market_chart_by_id(id_, vs_currency='USD', days='max')
  
    df_prices = pd.DataFrame(data['prices'], columns=['date', id_])
    df_prices = df_prices[df_prices[id_] > 0]
    df_prices['date'] = pd.to_datetime(df_prices['date'], unit='ms').dt.date
    df_prices['date'] = pd.to_datetime(df_prices['date'])
    df_prices = df_prices.set_index('date', drop=True)
    df_prices = df_prices.loc[~df_prices.index.duplicated(keep='first')]
    
    if len(df_prices) < 6 * 30:
        print(f'Excluding {symbol}, prices data available only for {len(df_prices)} < {6*30} days')
        exclude_list.append(id_)
        continue
    prices_data = pd.concat([prices_data, df_prices], axis=1)
    
    df_mcaps = pd.DataFrame(data['market_caps'], columns=['date', id_])
    df_mcaps = df_mcaps[df_mcaps[id_] > 0]
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'], unit='ms').dt.date
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'])
    df_mcaps = df_mcaps.set_index('date', drop=True)
    df_mcaps = df_mcaps.loc[~df_mcaps.index.duplicated(keep='first')]
    
    if len(df_mcaps) < 6 * 30:
        print(f'Note: {symbol}, marketcap data available only for {len(df_mcaps)} < {6 * 30} days')
        ## continue
    df_mcaps = df_mcaps.reindex(df_prices.index)
    marketcaps = pd.concat([marketcaps, df_mcaps], axis=1)
    
    

Excluding BTCB, prices data available only for 129 < 180 days


In [32]:
lt_passed = nq_passed[
    ~nq_passed.index.isin(exclude_list)
]

### Slippage check

To derive a slippage threshold, consider an abstract univ2-like pool with $5m\$$ overall liquidity. 
I.e if pool has reserves $R_1$, $R_2$ and $p_1$ and $p_2$ are the correspondent tokens prices (in USD), then
$$R_1p_1 = R_2p_2 = 2.5m \$$$

Swapping $x$ token1 for token2 will result in the output amount $y$ such that
$$R_1\cdot R_2 = (R_1 + x) \cdot (R_2 - y)$$
$$y = \frac{R_2\cdot x}{R_1 + x}$$
Then the delivery price will be
$$p_d = \frac{y}{x} = \frac{R_2}{R_1 + x}$$
The spot price may be estimated as a delivery price for relatively small input amount $x_0$
$$p_s = \frac{y_0}{x_0} = \frac{R_2}{R_1 + x_0}$$

Then the theoretical slippage for this pool will be
$$slippage = \frac{p_d}{p_s} - 1 = \frac{R_1+x_0}{R_1+x} - 1 = \frac{p_1R_1+p_1x_0}{p_1R_1+p_1x}-1$$

In [33]:
liq = 2*10**6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e3
trade_value = 1e5

In [34]:
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .5
max_slippage = round(2 * max_slippage, 2) / 2 
max_slippage

-0.09

In [35]:
url_0x = "https://avalanche.api.0x.org/swap/v1/price"

In [36]:
decimals = 18
slippages = []
for id_, data in lt_passed.iterrows():
    query = {
        'buyToken': 'WETH',
        'sellToken': coins[id_]['addr'],
        'sellAmount': int(trade_value_tiny / data['current_price'] * 10 ** decimals),
    }
    
    # spot price is calculated as a price for 1000$ swap
    resp = requests.get(url_0x, params=query)
    swap = resp.json()
    spot_price = float(swap['price'])
    
    query['sellAmount'] = int(trade_value / data['current_price'] * 10 ** decimals)
    resp = requests.get(url_0x, params=query)
    swap = resp.json()
    del_price = float(swap['price'])
    
    slippage = del_price / spot_price - 1
    
    slippages.append({
        'symb': id_,
        'spot_price': spot_price,
        'delivery_price': del_price,
        'slippage': slippage
    })
    
    

In [37]:
slippages_df = pd.DataFrame(slippages)
slippages_df

,symb,spot_price,delivery_price,slippage
0,avalanche-2,0.011495,0.011471,-0.002012
1,benqi-liquid-staked-avax,0.012013,0.011989,-0.001984
2,joe,0.000160,0.000156,-0.027952
3,benqi,0.000006,0.000005,-0.056058


In [38]:
exclude_list = list(slippages_df[abs(slippages_df['slippage']) > abs(max_slippage)]['symb'])
exclude_list

[]

In [39]:
st_passed = lt_passed[
    ~lt_passed.index.isin(exclude_list)
]

### Marketcap ranking & top10 filtering

In [40]:
chosen_tokens = st_passed.sort_values("market_cap", ascending=False).iloc[:10]

In [41]:
chosen_tokens

,name,symbol,market_cap,market_cap_rank,current_price,total_supply,circulating_supply,addr
id,,,,,,,,
avalanche-2,Avalanche,AVAX,5.368785e+09,17.0,17.970000,4.129870e+08,2.982377e+08,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
benqi-liquid-staked-avax,BENQI Liquid Staked AVAX,SAVAX,8.824765e+07,275.0,18.810000,4.691643e+06,4.691307e+06,0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be
joe,JOE,JOE,8.634458e+07,279.0,0.252443,4.396521e+08,3.419847e+08,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd
benqi,BENQI,QI,2.680867e+07,591.0,0.008875,7.200000e+09,3.024486e+09,0x8729438eb15e2c8b576fcc6aecda6a148776c0f5


In [42]:
chosen_tokens_symbols = list(chosen_tokens.index)
prices_data = prices_data[chosen_tokens_symbols]
marketcaps = marketcaps[chosen_tokens_symbols]

In [43]:
without_nan_index = (marketcaps.isnull().sum(axis=1) == 0) & (prices_data.isnull().sum(axis=1) == 0)
marketcaps = marketcaps[without_nan_index]
prices_data = prices_data[without_nan_index]

## Weighting 

**Weighting requirements**
* The maximum weight any one token can have is 50%.
* All excess weight is proportionally redistributed to all uncapped tokens. 
* Any asset with a weight below 0.5% will be removed from the index.


### Weight mcaps

In [44]:
# exclude savax from weighting since it should be a part of avax weight
mcaps = marketcaps[marketcaps.columns.drop("benqi-liquid-staked-avax")]
weights = mcaps.div(mcaps.sum(axis=1), axis=0)
weights = weights.sort_values(weights.last_valid_index(), axis=1, ascending=False)

In [45]:
weights

,avalanche-2,joe,benqi
date,,,
2022-03-04,0.985637,0.009707,0.004656
2022-03-05,0.986567,0.009406,0.004027
2022-03-06,0.986591,0.009341,0.004068
2022-03-07,0.986287,0.009534,0.004179
2022-03-08,0.986677,0.009240,0.004083
...,...,...,...
2022-10-24,0.980949,0.013854,0.005197
2022-10-25,0.980508,0.014214,0.005278
2022-10-26,0.981136,0.013756,0.005107


In [46]:
fig = px.line(weights,
              labels={'value': 'weight, %', 'variable': ''})
fig.update_traces(
    hovertemplate="%{y}"
)
fig.update_yaxes(
    tickformat=".2%",
)
fig.update_xaxes(
    showspikes=True,
    spikethickness=2,
    spikedash="dot",
    spikecolor="#999999",
    spikemode="across",
)
fig.update_layout(
    ## showlegend=False,
    hovermode="x",
    hoverdistance=100,  ## Distance to show hover label of data point
    spikedistance=1000,  ## Distance to show spike
    template='plotly_white',
    title='Weights without max constraint'
)

### Adjust weights

This process adjusts weights to adhere to the weight constraints described in our methodology. In addition the `remove_tiny_weights` function checks for any assets with a weight below the minimum threshold of $0.5\%$ and proportionally increases all remaining weights (which are $> 0.5\%$). Note that after this procedure readjusting weights to respect maximum may be needed.

In [47]:
def adjust_weights(weights, max_weight):
    w = weights.copy()
    
    while (w > max_weight).any(axis=None):
        w[w > max_weight] = max_weight
        c = 1 - w.sum(axis=1)
        w_less = w[w < max_weight]
        w[(w < max_weight) & (w > 0)] += w_less.div(w_less.sum(axis=1), axis=0).mul(c, axis=0)
    
    return w

In [48]:
def remove_tiny_weights(weights, min_weight):
    w = weights.copy()
    
    w[w < min_weight] = 0
    w = w.div(w.sum(axis=1), axis=0)
    
    return w

In [49]:
adjusted_weights = adjust_weights(weights, max_weight=0.5) 
adjusted_weights = remove_tiny_weights(adjusted_weights, min_weight=0.005)
adjusted_weights = adjust_weights(adjusted_weights, max_weight=0.5)

adjusted_weights = adjusted_weights.sort_values(adjusted_weights.last_valid_index(),ascending=False, axis = 1)
adjusted_weights

,avalanche-2,joe,benqi
date,,,
2022-03-04,0.5,0.337922,0.162078
2022-03-05,0.5,0.350109,0.149891
2022-03-06,0.5,0.348311,0.151689
2022-03-07,0.5,0.347621,0.152379
2022-03-08,0.5,0.346767,0.153233
...,...,...,...
2022-10-24,0.5,0.363603,0.136397
2022-10-25,0.5,0.364601,0.135399
2022-10-26,0.5,0.364624,0.135376


In [50]:
fig = px.line(adjusted_weights,
              labels={'value': 'weight, %', 'variable': ''})
fig.update_traces(
    hovertemplate="%{y}"
)
fig.update_yaxes(
    tickformat=".2%",
    range=[0,1],
)
fig.update_xaxes(
    showspikes=True,
    spikethickness=2,
    spikedash="dot",
    spikecolor="#999999",
    spikemode="across",
)
fig.update_layout(
    hovermode="x",
    hoverdistance=100,  ## Distance to show hover label of data point
    spikedistance=1000,  ## Distance to show spike
    template='plotly_white',
    title='Weights with max constraint'
)

### Split AVAX 

In [51]:
if 'avalanche-2' and 'benqi-liquid-staked-avax' in chosen_tokens.index:
    split = 0.4
    adjusted_weights['benqi-liquid-staked-avax'] = adjusted_weights['avalanche-2'] * split
    adjusted_weights['avalanche-2'] = adjusted_weights['avalanche-2'] * (1 - split)


### Convert weights to the [1, 255] scale

On the contract side weights are integer numbers from the interval $[1, 255]$ with total sum $255$, so it's needed to convert retrieved weights to this format.

Note that one can't just round weights to integers after scaling, since it doesn't guarantee that their sum will be $255$. To fix that firstly floor function is applied to the weights and then $1$ is being added to the $k$ weights with largest fractional parts.

In [52]:
def convert_weights(weights):
    w_scaled = weights * 255
    w_res = np.floor(w_scaled).astype(int)    
    remainders = w_scaled - w_res    
    k = round(remainders.sum())
    w_below_max = w_res[w_res<128]
    for i in range(k):
        w_below_max[i] +=1
    for i in w_below_max.index:
        w_res[i] = w_below_max[i]
    return w_res

In [53]:
last_weights = adjusted_weights.iloc[-1].dropna()

In [54]:
converted_last_weights = convert_weights(last_weights)

## Summary table

In [55]:
cai = pd.DataFrame()
cai.index = chosen_tokens.index
cai['name'] = chosen_tokens['name']
cai['symbol'] = chosen_tokens['symbol']
cai['market_cap'] = marketcaps.iloc[-1].astype(int)
cai['price'] = prices_data.iloc[-1]
cai['weight'] = last_weights
cai['weight_converted'] = converted_last_weights
cai['address'] = chosen_tokens['addr']

cai = cai[cai['weight'] > 0]
cai = cai.sort_values("market_cap", ascending=False)
cai

,name,symbol,market_cap,price,weight,weight_converted,address
id,,,,,,,
avalanche-2,Avalanche,AVAX,5052552433,16.940762,0.300000,77,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
joe,JOE,JOE,83422962,0.243732,0.382645,98,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd
benqi-liquid-staked-avax,BENQI Liquid Staked AVAX,SAVAX,83266977,17.708811,0.200000,51,0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be
benqi,BENQI,QI,25585425,0.008446,0.117355,29,0x8729438eb15e2c8b576fcc6aecda6a148776c0f5


In [56]:
print(cai[['address', 'weight_converted']].to_csv(None, index=False))

address,weight_converted
0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7,77
0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd,98
0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be,51
0x8729438eb15e2c8b576fcc6aecda6a148776c0f5,29

